In [1]:
import os
# Find the latest version of spark 3.0  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.3.1'
spark_version = 'spark-3.3.1'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q https://downloads.apache.org/spark/spark-3.3.1/spark-3.3.1-bin-hadoop2.tgz
!tar xf spark-3.3.1-bin-hadoop2.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2"

# Start a SparkSession
import findspark
findspark.init()

Get:1 https://cloud.r-project.org/bin/linux/ubuntu focal-cran40/ InRelease [3,622 B]
Ign:2 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu2004/x86_64  InRelease
Hit:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2004/x86_64  InRelease
Get:4 http://security.ubuntu.com/ubuntu focal-security InRelease [114 kB]
Hit:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu2004/x86_64  Release
Hit:6 http://archive.ubuntu.com/ubuntu focal InRelease
Hit:7 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu focal InRelease
Get:8 http://archive.ubuntu.com/ubuntu focal-updates InRelease [114 kB]
Hit:9 http://ppa.launchpad.net/cran/libgit2/ubuntu focal InRelease
Get:11 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu focal InRelease [18.1 kB]
Get:12 http://archive.ubuntu.com/ubuntu focal-backports InRelease [108 kB]
Hit:13 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu focal InRelease
Get:14 http://security.ubuntu.com/ubuntu fo

In [2]:
# Start Spark session
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("vine_review_analysis").getOrCreate()

In [3]:
# Read in data
from pyspark import SparkFiles
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Beauty_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
df = spark.read.option("encoding", "UTF-8").csv(SparkFiles.get(""), sep="\t", header=True, inferSchema=True)
df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+---------------------+-------------------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|          review_body|        review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+---------------------+-------------------+
|         US|    1797882|R3I2DHQBR577SS|B001ANOOOE|       2102612|The Naked Bee Vit...|          Beauty|          5|            0|          0|   N|                Y|          Five Stars| Love this, excell...|2015-08-31 00:00:00|
|         US|   18381298|R1QNE9NQFJC2Y4|B0016J22EQ|     106393691|Alba Botanica Sun.

In [4]:
# Retrieve all the rows where the total_votes count is equal to or greater than 20
votes_df = df.filter("total_votes>=20")
votes_df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-------------------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|        review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-------------------+
|         US|   22393078|R2QRUE9REK8OUC|B00461F4PA|     608719013|Baby Foot Exfolia...|          Beauty|          5|           18|         23|   N|                Y|Better Than Any Pedi|First off, I'll s...|2015-08-31 00:00:00|
|         US|   46053231|R2POXRW6PXHXZG|B00OCJZ0GE|     717345177|Bed Head Little T...| 

In [7]:
helpfulVotes_df = votes_df.filter("helpful_votes/total_votes>=0.5")
helpfulVotes_df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-------------------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|        review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-------------------+
|         US|   22393078|R2QRUE9REK8OUC|B00461F4PA|     608719013|Baby Foot Exfolia...|          Beauty|          5|           18|         23|   N|                Y|Better Than Any Pedi|First off, I'll s...|2015-08-31 00:00:00|
|         US|   46053231|R2POXRW6PXHXZG|B00OCJZ0GE|     717345177|Bed Head Little T...| 

In [10]:
vineYes_df = helpfulVotes_df.filter("vine=='Y'")
vineYes_df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-------------------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|        review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-------------------+
|         US|   47770601|R106V6GUNWRSSS|B00YAR86BY|     136411552|Philips Sonicare ...|          Beauty|          5|          857|        889|   Y|                N|Improved build, g...|According to spec...|2015-08-27 00:00:00|
|         US|   52846213|R37N8KZS48W36A|B00YJJXJ06|     791197070|Maybelline New Yo...| 

In [11]:
vineNo_df = helpfulVotes_df.filter("vine=='N'")
vineNo_df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-------------------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|        review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-------------------+
|         US|   22393078|R2QRUE9REK8OUC|B00461F4PA|     608719013|Baby Foot Exfolia...|          Beauty|          5|           18|         23|   N|                Y|Better Than Any Pedi|First off, I'll s...|2015-08-31 00:00:00|
|         US|   46053231|R2POXRW6PXHXZG|B00OCJZ0GE|     717345177|Bed Head Little T...| 

In [17]:
# Import functions
from pyspark.sql.functions import count
# Determine the total number of paid reviews
vineYes_df.select(count("review_id")).show()

+----------------+
|count(review_id)|
+----------------+
|             647|
+----------------+



In [18]:
# Determine the total number of unpaid reviews
vineNo_df.select(count("review_id")).show()

+----------------+
|count(review_id)|
+----------------+
|           74113|
+----------------+



In [23]:
# Determine the the number of 5-star paid reviews
fiveStarPaid_reviews = vineYes_df.filter("star_rating==5").select(count("star_rating")).show()

+------------------+
|count(star_rating)|
+------------------+
|               229|
+------------------+



In [24]:
# Determine the the number of 5-star unpaid reviews
fiveStarUnpaid_reviews = vineNo_df.filter("star_rating==5").select(count("star_rating")).show()

+------------------+
|count(star_rating)|
+------------------+
|             43217|
+------------------+



In [38]:
# Determine the percentage of 5-star paid reviews
#starRating_count = vineYes_df.select(count("star_rating"))
percentage_paid = float(229) / float(647)
percentage_paid


0.35394126738794435

In [39]:
# Determine the percentage of 5-star unpaid reviews
percentage_unpaid = float(43217) / float(74113)
percentage_unpaid

0.5831230688273312